## Interactive distributions

Adjust the sliders to change the mean (μ) and standard deviation (σ) of the distribution.

The mean moves the distribution so that its mean is higher or lower

The standard deviation changes the spread, changing the uncertainty of the belief

## Play with the distributions

Use the sliders to generate distributions that represent different prior beliefs for theta:

  - *Certain belief that the theta is high*

  - *Certain belief that the theta is low* 
  
  - *Uncertain belief that the theta is high*   
  
  - *Uncertain belief that the theta is low* 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian(mu=0.0, sigma=1.0):
    # Fixed x range
    x = np.linspace(-20, 20, 1000)
    y = norm.pdf(x, mu, sigma)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')
    ax.fill_between(x, y, color='plum', alpha=0.4)
    ax.set_title("Gaussian Distribution")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_xlim(-20, 20)
    ax.set_ylim(0, 3)
    ax.grid(True)
    ax.legend()
    plt.show()

interact(
    plot_gaussian,
    mu=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description="σ")
);

interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

## Interpreting probability distributions

There are intuitive ways to read off probabilities from probability distributions like these.

According to the belief encoded by the distribution, the probability of theta being larger than 0.5 is the area under the curve above this value

Similarly, the probability of theta being between 0.1 and 0.7 is the area under the curve between these two values

Play with the sliders to see how the area under the curve changes as you change the minimum and maximum values.

The shaded area represents the probability of theta being in the range you selected.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian(mu=0.0, sigma=1.0, x_min=-1.0, x_max=1.0):
    # Fixed x range
    x = np.linspace(-20, 20, 1000)
    y = norm.pdf(x, mu, sigma)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')

    # Fill full curve lightly
    ax.fill_between(x, y, color='plum', alpha=0.2)

    # Highlight area between x_min and x_max
    mask = (x >= x_min) & (x <= x_max)
    ax.fill_between(x[mask], y[mask], color='mediumvioletred', alpha=0.6,
                    label=f"P({x_min:.2f} < θ < {x_max:.2f}) ≈ {norm.cdf(x_max, mu, sigma) - norm.cdf(x_min, mu, sigma):.2f}")

    ax.set_title("Probability as Area Under the Curve")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_xlim(-20, 20)
    ax.set_ylim(0, max(y) * 1.1)
    ax.grid(True)
    ax.legend()
    plt.show()

interact(
    plot_gaussian,
    mu=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description="σ"),
    x_min=widgets.FloatSlider(min=-10, max=10, step=0.1, value=-1.0, description="x min"),
    x_max=widgets.FloatSlider(min=-10, max=10, step=0.1, value=1.0, description="x max")
)


interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

<function __main__.plot_gaussian(mu=0.0, sigma=1.0, x_min=-1.0, x_max=1.0)>

## Bayesian credibility intervals

A Bayesian credibility interval is a range of values that contains the true value of the parameter with a certain probability. 

It is similar to a confidence interval in frequentist statistics, but it is based on the posterior distribution of the parameter rather than the sampling distribution.

Its interpretation is actually what most people think of when they hear the term "confidence interval". Its what they are looking for when they ask for a confidence interval.

You can construct a Bayesian credibility interval by picking the probability you want to contain the true value of the parameter, and then finding the range of values that contains that probability. 

Play with the sliders to see how the credibility interval changes as you change the probability.

The most typical is the 95% credibility interval $BCI_{95}$, which contains the value of the parameter with 95% probability.  

The 50% credibility interval $BCI_{50}$ would contains the value of the parameter with 50% probability. And so on.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian_bci(mu=0.0, sigma=1.0, bci=95.0):
    # Fixed x range
    x = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
    y = norm.pdf(x, mu, sigma)

    # Compute the central credible interval bounds
    alpha = 1 - bci / 100
    lower_bound = norm.ppf(alpha / 2, mu, sigma)
    upper_bound = norm.ppf(1 - alpha / 2, mu, sigma)

    # Max density for horizontal line
    y_bound = norm.pdf([lower_bound, upper_bound], mu, sigma).min()

    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot the density curve
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')

    # Shade full area lightly
    ax.fill_between(x, y, color='plum', alpha=0.2)

    # Shade credible interval
    mask = (x >= lower_bound) & (x <= upper_bound)
    ax.fill_between(x[mask], y[mask], color='mediumvioletred', alpha=0.6,
                    label=f"{bci:.0f}% BCI: [{lower_bound:.2f}, {upper_bound:.2f}]")

    # Draw horizontal line intersecting the interval ends
    ax.hlines(y=y_bound, xmin=lower_bound, xmax=upper_bound,
              color='black', linestyle='--', linewidth=1)

    # Add vertical lines at bounds
    ax.vlines([lower_bound, upper_bound], ymin=0, ymax=y_bound,
              color='black', linestyle='--', linewidth=1)

    # Labels and legend
    ax.set_title("Central Bayesian Credible Interval (BCI)")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_xlim(mu - 4*sigma, mu + 4*sigma)
    ax.set_ylim(0, max(y) * 1.1)
    ax.grid(True)
    ax.legend()
    plt.show()

interact(
    plot_gaussian_bci,
    mu=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description="σ"),
    bci=widgets.FloatSlider(min=50, max=99, step=1, value=95, description="BCI (%)")
)


interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

<function __main__.plot_gaussian_bci(mu=0.0, sigma=1.0, bci=95.0)>

## Lets set a prior belief for theta

We need to think carefully about what theta means

Theta is representing cognitive ability in our go-no-go task

0 is the worst possible cognitive ability, and 1 is the best possible cognitive ability.

So we need to set a prior distribution for theta that is between 0 and 1.

The distributions are above do not do this so lets fix this. 

We can use a beta distribution to represent our prior belief about theta.

The beta distribution is a continuous probability distribution defined on the interval [0, 1] so it is perfect for our needs.

Play around with beta distribution parameters to see how the shape of the distribution changes.

- **You are complete uncertain about the ability**  
  → Set the highest spread you can 
  - widest possible prior

- **You are quite certain that ability will be high**  
  → Set a **narrow** prior centered on a **high value**

- **You are uncertain but you think ability is low**  
  → Set a **wide** prior centered on a **low value**


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

def plot_beta(k, n):
    if k > n:
        print("Error: k must be ≤ n.")
        return
    x = np.linspace(0, 1, 1000)
    alpha_param = k + 1
    beta_param = (n - k) + 1
    y = beta.pdf(x, alpha_param, beta_param)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y, label=f"Beta({alpha_param}, {beta_param})", color="blue")
    ax.fill_between(x, y, color="skyblue", alpha=0.4)
    ax.set_title("Distribution of θ")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    plt.show()

# Create sliders
n_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n")
k_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k")

# Callback to update the max value of k_slider when n changes
def update_k_slider(*args):
    k_slider.max = n_slider.value
    if k_slider.value > k_slider.max:
        k_slider.value = k_slider.max

n_slider.observe(update_k_slider, names='value')

# Display everything
ui = widgets.VBox([n_slider, k_slider])
out = widgets.interactive_output(plot_beta, {'k': k_slider, 'n': n_slider})

display(ui, out)


Output()